# Check GPU

In [1]:
!nvidia-smi

Wed Jul  5 05:25:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install Dependencies

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq accelerate==0.20.3 --progress-bar off
!pip install -qqq bitsandbytes==0.39.0 --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq evaluate==0.4.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq rouge_score==0.1.2 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.30.2 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.


# Import Libraries

In [3]:
import os

import evaluate
import pandas as pd
import torch
import transformers
from datasets import Dataset
from IPython.display import display
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

OUTPUT_DIR, DASH_LINE, HUGGINGFACE_API_KEY, CLASSIFICATION = (
    "experiments",
    "-" * 100,
    os.environ["API_KEY"],
    1,
)
MODEL_NAME = [
    "tiiuae/falcon-7b",
    "tiiuae/falcon-7b-instruct",
    "tiiuae/falcon-rw-1b",
    "EleutherAI/gpt-neo-125m",
    "bigscience/bloom-1b7",
][1]
PEFT_MODEL = f"lyoh001/{MODEL_NAME.split('/')[-1]}-qlora"


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib'), PosixPath('/usr/local/lib/x86_64-linux-gnu'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


# Load Datasets

In [4]:
df = pd.read_csv(url, delimiter=",", na_values="", index_col=None)
display(df)


,question,answer
0,#Intern #US #TATTOO #Wisconsin #Ohio #NC #PA #Florida #Colorado #Iowa #Nevada #Virginia #NV #mlb Travel Destinations---&gt;http://t.co/TIHBJKF2,not disaster
1,RT @Jack4Ward: Get in on the fun every Thursday with the @csindependent #FunBrief http://ow.ly/br9Wi #CoSprings #Colorado,not disaster
2,Welcome to our newest STUDENTathlete- Reagan Biechler from Colorado Springs (CO) ’13- Baseball. http://t.co/lzeiYMpq,not disaster
3,Denver Post: #Colorado governor signs bill creating rules for public access to ballots http://t.co/Hmp1wQ8P #opengov,not disaster
4,"Pretty sure I'm going to live in Manitou Springs, CO someday. Maybe live here and work in Colorado Springs?? #theplan",not disaster
...,...,...
283876,"Bahamians rescued victims of Hurricane Dorian with jet skis and a bulldozer as the U.S. Coast Guard, Britain’s Royal Navy and a handful of aid groups tried to get food and medicine to survivors and take the most desperate people to safety.\nhttps://www.politico.com/story/2019/09/03/hurricane-dorian-bahamas-1479992 …",hurricane
283877,"""Bahamians rescued victims of Hurricane Dorian with Jet Skis and a bulldozer as the U.S. Coast Guard, Britain’s Royal Navy and a handful of aid groups tried to get food and medicine to survivors"" Deadly Hurricane Dorian - Los Angeles Times https://www.latimes.com/world-nation/story/2019-09-03/hurricane-dorian-bahamas-east-coast-evacuations …",hurricane
283878,"Hurricane Dorian ravaged Bahamas cries for international help; thousands without food, homes, and medicine and stuck in floodwaters. Will Trump depute a new agency to coordinate?\nhttps://www.ibtimes.com/hurricane-dorian-relief-how-help-bahamas-after-hurricane-destruction-2821834 …\n#Bahamas #TRUMP2020Landside #StockMarket #dorianhurricane",hurricane
283879,"Bahamians rescue victims of Hurricane Dorian with jet skis and a bulldozer as the US Coast Guard, Britain's Royal Navy and aid groups try to get food and medicine to survivors https://www.trtworld.com/americas/bahamians-begin-rescues-as-dorian-moves-on-toward-us-coast-29508 …",hurricane


# Train, Validation, Test Split

In [5]:
train_size, test_size = 0.7, 0.5

if CLASSIFICATION:
    class_distribution = df["answer"].value_counts(normalize=True)
    train_indices, test_indices, val_indices, temp_indices = [], [], [], []

    for class_label in class_distribution.index:
        class_indices = df[df["answer"] == class_label].index.tolist()
        split_index = int(len(class_indices) * train_size)
        train_indices.extend(class_indices[:split_index])
        temp_indices.extend(class_indices[split_index:])

    for class_label in class_distribution.index:
        class_indices = [
            index for index in temp_indices if df["answer"][index] == class_label
        ]
        split_index = int(len(class_indices) * test_size)
        test_indices.extend(class_indices[:split_index])
        val_indices.extend(class_indices[split_index:])

    df_train = df.loc[train_indices]
    df_val = df.loc[val_indices]
    df_test = df.loc[test_indices]

    print(f"Total count:\n{df['answer'].value_counts(normalize=True)}")
    print(DASH_LINE)
    print(f"Train count:\n{df_train['answer'].value_counts(normalize=True)}")
    print(DASH_LINE)
    print(f"Val count:\n{df_val['answer'].value_counts(normalize=True)}")
    print(DASH_LINE)
    print(f"Test count:\n{df_test['answer'].value_counts(normalize=True)}")
    print(DASH_LINE)


else:
    split_index = int(len(df) * train_size)
    df_train = df[:split_index]
    temp_df = df[split_index:]

    split_index = int(len(temp_df) * test_size)
    df_test = temp_df[:split_index]
    df_val = temp_df[split_index:]

print(f"Total shape:\t{df.shape}")
print(f"Train shape:\t{df_train.shape}")
print(f"Val shape:\t{df_val.shape}")
print(f"Test shape:\t{df_test.shape}")


Total count:
not disaster    0.406477
hurricane       0.232057
flood           0.127382
earthquake      0.075899
tornado         0.058331
wildfire        0.051913
blizzard        0.023672
typhoon         0.006746
haze            0.005626
meteorite       0.003991
collapse        0.003209
meteor          0.002603
cyclone         0.001413
pandemic        0.000514
volcano         0.000166
Name: answer, dtype: float64
----------------------------------------------------------------------------------------------------
Train count:
not disaster    0.406491
hurricane       0.232064
flood           0.127383
earthquake      0.075900
tornado         0.058332
wildfire        0.051910
blizzard        0.023673
typhoon         0.006744
haze            0.005621
meteorite       0.003991
collapse        0.003206
meteor          0.002602
cyclone         0.001409
pandemic        0.000513
volcano         0.000161
Name: answer, dtype: float64
-----------------------------------------------------------------

# Load the Model

In [6]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, params in model.named_parameters():
        all_params += params.numel()
        if params.requires_grad:
            trainable_params += params.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_params} || trainable%: {100 * trainable_params / all_params}"
    )


def inference(user_input, max_new_tokens=1, temperature=0.1, top_p=0.7):
    prompt = f"""Your task is to run a text classification on the natural disaster tweets.
    <question>: {user_input}
    <answer>: 
    """.strip()

    encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    generation_config = model.generation_config
    generation_config.max_new_tokens = max_new_tokens
    generation_config.temperature = temperature
    generation_config.top_p = top_p
    generation_config.num_return_sequences = 1
    generation_config.pad_token_id = tokenizer.eos_token_id
    generation_config.eos_token_id = tokenizer.eos_token_id
    print(generation_config)
    print(DASH_LINE)
    print(f"<encoded prompt>: {encoding['input_ids'][0]}")
    print(DASH_LINE)
    print(
        f"<decoded prompt>: {tokenizer.decode(encoding['input_ids'][0], skip_special_tokens=True)}"
    )
    print(DASH_LINE)
    with torch.inference_mode():
        output = model.generate(
            input_ids=encoding["input_ids"],
            attention_mask=encoding["attention_mask"],
            generation_config=generation_config,
            do_sample=False,
            use_cache=False,
        )
    payload = tokenizer.decode(output[0], skip_special_tokens=True)
    start = payload.find("<answer>: ") + len("<answer>: ")
    end = payload.find("<", start)
    payload = payload[start:end].strip()
    print(f"<answer>: {payload}")
    return payload


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, config)
print_trainable_parameters(model)


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


# Display Dataset

In [ ]:
display(df.sample(10))


# Inference Before Training

In [ ]:
%%time
inference(
    input("<prompt>: "),
    max_new_tokens=[1, 50, 100, 200][1],
    temperature=[0.1, 0.7, 1.0][0],
    top_p=0.7,
)

# Build the Dataset

In [7]:
def generate_prompt(data_point):
    return f"""Your task is to run a text classification on the natural disaster tweets.
    <question>: {data_point["question"]}
    <answer>: {data_point["answer"]}
    """.strip()


def generate_and_tokenize_prompt(data_point):
    return tokenizer(generate_prompt(data_point), padding=True, truncation=True)


data_train = Dataset.from_pandas(df_train)
data_test = Dataset.from_pandas(df_test)
data_val = Dataset.from_pandas(df_val)
data_train = data_train.shuffle().map(generate_and_tokenize_prompt)
data_test = data_test.shuffle().map(generate_and_tokenize_prompt)
data_val = data_val.shuffle().map(generate_and_tokenize_prompt)
print(data_train, data_test, data_val)


Map:   0%|          | 0/198708 [00:00<?, ? examples/s]

Map:   0%|          | 0/42582 [00:00<?, ? examples/s]

Map:   0%|          | 0/42588 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 198708
}) Dataset({
    features: ['question', 'answer', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 42582
}) Dataset({
    features: ['question', 'answer', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 42588
})


# Toggling Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

# Training

In [8]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)
trainer = transformers.Trainer(
    model=model,
    train_dataset=data_train,
    eval_dataset=data_val,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,4.549000
2,4.304800
3,3.969600
4,4.517900
5,4.283100
6,3.960100
7,3.691600
8,3.944100
9,3.880500
10,4.103500


TrainOutput(global_step=80, training_loss=2.432806073129177, metrics={'train_runtime': 473.1721, 'train_samples_per_second': 0.676, 'train_steps_per_second': 0.169, 'total_flos': 353724743583744.0, 'train_loss': 2.432806073129177, 'epoch': 0.0})

# Run Model Performance

In [9]:
rouge = evaluate.load("rouge")
df_test_sample = df_test.sample(100, random_state=11)
questions, original_answers = df_test_sample["question"], df_test_sample["answer"]
model_answers = [
    inference(
        question,
        max_new_tokens=[1, 50, 100, 200][1],
        temperature=[0.1, 0.7, 1.0][0],
        top_p=0.7,
    )
    for question in questions
]
model_results = rouge.compute(
    predictions=model_answers,
    references=original_answers.values,
    use_aggregator=True,
    use_stemmer=True,
)
print(model_results)
display(
    pd.DataFrame(
        list(zip(original_answers, model_answers)),
        columns=["original_answers", "model_answers"],
    )
)


GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 50,
  "pad_token_id": 11,
  "temperature": 0.1,
  "top_p": 0.7,
  "transformers_version": "4.30.2"
}

----------------------------------------------------------------------------------------------------
<encoded prompt>: tensor([ 4560,  4957,   304,   271,  1257,   241,  2288, 17873,   313,   248,
         2616, 11893, 26603,    25,   561,    39, 27465, 48190,   204,   428,
        29421, 11481, 16697,   836,  2233,   299, 21749, 36326,    39, 46617,
        48190], device='cuda:0')
----------------------------------------------------------------------------------------------------
<decoded prompt>: Your task is to run a text classification on the natural disaster tweets.
    <question>:  jasminehuerta i hope you drown 
    <answer>:
----------------------------------------------------------------------------------------------------


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


<answer>: not disaster related
GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 50,
  "pad_token_id": 11,
  "temperature": 0.1,
  "top_p": 0.7,
  "transformers_version": "4.30.2"
}

----------------------------------------------------------------------------------------------------
<encoded prompt>: tensor([ 4560,  4957,   304,   271,  1257,   241,  2288, 17873,   313,   248,
         2616, 11893, 26603,    25,   561,    39, 27465, 48190,   204,    14,
        19595,  7184,    37, 53372,  1736,   204,    18,  4736,   851,    18,
         1676,    18,    94, 33076,    37,  3314,   317, 24471, 32213,  6199,
         2203,   271,   717,  1547,   275,   248, 29777, 34502, 23944, 19037,
         7561,    82,   443,    25,  2974,    26, 33268,    26,    27,    78,
        38988,    53,   561,    39, 46617, 48190], device='cuda:0')
----------------------------------------------------------------------------------------------------

,original_answers,model_answers
0,not disaster,not disaster related
1,tornado,tornado
2,hurricane,hurricane
3,earthquake,earthquake
4,not disaster,not disaster related
...,...,...
95,meteorite,meteorite
96,not disaster,not disaster related
97,hurricane,hurricane
98,flood,not disaster related


# Display Dataset

In [10]:
display(df.sample(10))


,question,answer
243047,I can't believe he stayed this close without taking shelter! Incredible tornado video! http://ow.ly/4J51b,tornado
14243,"What's New! Russia cleans up after meteor blast injures more than 1,000 http://t.co/egnUSFRZ",meteorite
68268,<HASHTAG> floortest is happening on a <HASHTAG> saturday! lord shani is known as the chief justice of universe! during sadesati <USER>,not disaster
95362,va - <NUMBER> greatest hits vol <NUMBER> - <NUMBER> » full version software  <REPEAT> : we do not place on your site files and try to  <REPEAT> <URL>,not disaster
126338,"another up beat <HASHTAG> song "" i don't want no scrub, a scrub is a guy that can't get no love from me"" @tlc",not disaster
90667,heartbreaking. sophia bush's ex dan fredinburg died following the nepal earthquake today: <URL> http://t.co/â°ã_,earthquake
133020,"reuters|hurricane maria brings destruction, heavy floods to puerto rico| <URL> <URL>",hurricane
184896,<USER> i saw on the road  <REPEAT> what abt you? putting k.stew in the same category as j.law when you've only seen <NUMBER> of her movie  <REPEAT>,not disaster
150808,here's the bet: take whatever money you were planning on dangling in front of me & then donate it to oklahoma relief and t…,tornado
48902,@kattalyst_ so don't be silly lol,not disaster


# Inference

In [11]:
%%time
inference(
    input("<prompt>: "),
    max_new_tokens=[1, 50, 100, 200][1],
    temperature=[0.1, 0.7, 1.0][0],
    top_p=0.7,
)


<prompt>:  here's the bet: take whatever money you were planning on dangling in front of me & then donate it to oklahoma relief and t…	


GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 50,
  "pad_token_id": 11,
  "temperature": 0.1,
  "top_p": 0.7,
  "transformers_version": "4.30.2"
}

----------------------------------------------------------------------------------------------------
<encoded prompt>: tensor([ 4560,  4957,   304,   271,  1257,   241,  2288, 17873,   313,   248,
         2616, 11893, 26603,    25,   561,    39, 27465, 48190,   967,    18,
           94,   248,   711,    37,   980,  4648,  1569,   299,   646,  4042,
          313, 55787,   272,  2365,   275,   454,   204,    17,   832, 17902,
          334,   271,  4469,    87, 13769,  7886,   273,   240,   898,   192,
          561,    39, 46617, 48190], device='cuda:0')
----------------------------------------------------------------------------------------------------
<decoded prompt>: Your task is to run a text classification on the natural disaster tweets.
    <question>: here's the bet

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


<answer>: tornado
CPU times: user 31.9 s, sys: 188 ms, total: 32.1 s
Wall time: 2min 1s


'tornado'

# Display Performance

In [14]:
print(f"rougeLsum score: {model_results['rougeLsum']}")
display(
    pd.DataFrame(
        list(zip(original_answers, model_answers)),
        columns=["original_answers", "model_answers"],
    )
)


rougeLsum score: 0.821333333333333


,original_answers,model_answers
0,not disaster,not disaster related
1,tornado,tornado
2,hurricane,hurricane
3,earthquake,earthquake
4,not disaster,not disaster related
5,earthquake,earthquake
6,flood,flood
7,hurricane,hurricane
8,not disaster,not disaster related
9,tornado,tornado


# Save Trained Model

In [ ]:
model.save_pretrained("trained_model")
model.push_to_hub(
    PEFT_MODEL,
    use_auth_token=HUGGINGFACE_API_KEY,
)
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    device_map="auto",
    torch_dtype=torch.bfloat16
    if torch.cuda.get_device_capability()[0] == 8
    else torch.float16,
    trust_remote_code=True,
)
model = PeftModel.from_pretrained(model, PEFT_MODEL)
model = model.merge_and_unload()
model.push_to_hub(f"{PEFT_MODEL}-merged", use_auth_token=HUGGINGFACE_API_KEY)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.push_to_hub(f"{PEFT_MODEL}-merged", use_auth_token=HUGGINGFACE_API_KEY)


# Inference from HuggingFace

In [ ]:
import time

import requests

API_URL = (
    "https://api-inference.huggingface.co/models/lyoh001/gpt-neo-125m-qlora-merged"
)
headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}


def get_payload(user_input):
    response = requests.post(API_URL, headers=headers, json=user_input)
    return response.json()[0]["generated_text"]


def retry_api_call(user_input):
    payload = None
    while not payload:
        try:
            payload = get_payload(user_input)
        except Exception as e:
            print(f"Error: {e}\nRetrying in 5 seconds.")
            time.sleep(5)
    return payload


output = retry_api_call(
    {
        "inputs": f"{input('')}",
        "parameters": {
            "max_new_tokens": [1, 50, 100, 200][-1],
            "temperature": [0.1, 0.7, 1.0][1],
            "top_p": 0.7,
        },
    }
)
print(output)


# Install AWS Dependencies

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install "sagemaker>=2.163.0" "boto3" "huggingface_hub==0.13.0" "hf-transfer" --upgrade --quiet
!pip install awscli --upgrade --quiet


# Deploy Sagemaker

In [ ]:
import json

import boto3
import sagemaker
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="AmazonSageMakerFullAccess")["Role"]["Arn"]

huggingface_model = HuggingFaceModel(
    image_uri=get_huggingface_llm_image_uri("huggingface", version="0.8.2"),
    env={
        "HF_MODEL_ID": f"lyoh001/{PEFT_MODEL}-merged",
        "SM_NUM_GPUS": json.dumps([1, 4][0]),
    },
    role=role,
)
predictor = huggingface_model.deploy(
    endpoint_name=f"{PEFT_MODEL}-merged-endpoint",
    instance_type=["ml.g5.2xlarge", "ml.g5.12xlarge"][0],
    initial_instance_count=1,
    container_startup_health_check_timeout=600,
)


# Authentication

In [ ]:
import boto3
from sagemaker import Session
from sagemaker.huggingface.model import HuggingFacePredictor

sess = Session(
    boto_session=boto3.Session(
        aws_access_key_id=input("AWS Access Key ID: "),
        aws_secret_access_key=input("AWS Secret Access Key: "),
        region_name=input("AWS Region: "),
    )
)
predictor = HuggingFacePredictor(
    endpoint_name=f"{PEFT_MODEL}-merged-endpoint",
    sagemaker_session=sess,
)


# Inference from Sagemaker

In [ ]:
predictor.predict(
    {
        "inputs": f"""Your task is to run a text classification on the natural disaster tweets.
        <question>: {input('')}
        <answer>: """,
        # "inputs": f"{input('')}",
        "parameters": {
            # "max_new_tokens": [1, 10, 100, 200, 1000][-1],
            "max_new_tokens": [1, 10, 100, 200][1],
            "temperature": [0.1, 0.7, 1.0][0],
            "top_p": 0.7,
        },
    }
)

# Delete Sagemaker

In [ ]:
# predictor.delete_model()
# predictor.delete_endpoint()
